In [11]:
import subprocess
import sys
import os
import time
import torch
import numpy as np
#torch.serialization.add_safe_globals([np.ndarray])

def check_gpu():
    """Print whether PyTorch sees a GPU, and if so, which GPU is in use."""
    print("PyTorch sees CUDA:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA version:", torch.version.cuda)
        print("PyTorch version:", torch.__version__)
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i} name:", torch.cuda.get_device_name(i))
            
def run_training():
    cmd = [
        sys.executable, os.path.join("yolov7", "train.py"),  # Use the current Python interpreter; adjust the path if needed
        "--workers", "2",
        "--device", "0",
        "--batch-size", "4",
        "--epochs", "2000",
        "--data", "pothole_dataset.yaml",  # Path to your data config YAML (update if needed)
        "--weights", "",
        "--cfg", os.path.join("cfg", "training", "yolov7-tiny.yaml"),
        "--name", "pothole_detector_yolov7",
        "--img-size", "416"
    ]
    
    print("Starting YOLOv7 training...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Training failed:")
        print(result.stderr)
        sys.exit(result.returncode)
    else:
        print("Training finished successfully.")

def run_detection(source):
    """
    Runs YOLOv7 detection/inference using the trained model.
    Adjust parameters as needed.
    """
    cmd = [
        sys.executable, os.path.join("yolov7", "detect.py"),
        "--weights", os.path.join("runs", "train", "pothole_detector_yolov7", "weights", "best.pt"),
        "--conf", "0.40",
        "--source", source,  # Path to image(s) or video file
        "--img-size", "416",
        "--name", "pothole_detector_inference",
        "--save-txt",
        "--save-conf"
    ]
    
    print("Starting YOLOv7 detection on source:", source)
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Detection failed:")
        print(result.stderr)
        sys.exit(result.returncode)
    else:
        print("Detection finished successfully.")

if __name__ == "__main__":
    # Optionally, change the working directory to the YOLOv7 repository if your script is outside of it:
    # os.chdir("path/to/yolov7")\
    
    # 1. Quick GPU check
    check_gpu()
    
    #run_training()
    
    # After training, run detection on the test set (update the path as needed)
    test_source = os.path.join("yolo_formatted_data", "images", "test")
    run_detection(test_source)


PyTorch sees CUDA: True
CUDA version: 12.6
PyTorch version: 2.7.0.dev20250116+cu126
GPU 0 name: NVIDIA GeForce RTX 3050 Laptop GPU
Starting YOLOv7 detection on source: yolo_formatted_data\images\test
Namespace(weights=['runs\\train\\pothole_detector_yolov7\\weights\\best.pt'], source='yolo_formatted_data\\images\\test', img_size=416, conf_thres=0.4, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='pothole_detector_inference', exist_ok=False, no_trace=False)
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

1 pothole, Done. (19.9ms) Inference, (59.7ms) NMS
 The image with the result is saved in: runs\detect\pothole_detector_inference\test_image_0.jpg
1 pothole, Done. (0.0ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\pothole_detector_inference\test_image_1.jpg
1